In [17]:
%load_ext lab_black

In [18]:
from pyprojroot import here
import os
from skimpy import clean_columns
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap
import joblib
from joblib import load

path_data = here("./data")
os.chdir(path_data)

data = pd.read_csv("data_pmpm.csv")
data_predict = data.drop(columns=["Hospital ID", "Per Member Per Month Cost"])
# Load the model
model = joblib.load("model_drivers.joblib")

Get the actual percentage of members with ER visits in the last 30 days for each hospital

Show them their average percentages for all four variables.

Add some noise for the features by Hospital ID

Then create a function to allow for simulated data with the exact percentage that someone shows

In [19]:
data.groupby("Hospital ID").mean().reset_index().round(2)

,Hospital ID,High Blood Pressure,High Cholesterol,Diabetes,Preventative Services,Per Member Per Month Cost
0,1,0.29,0.41,0.25,0.51,5123.60
1,2,0.28,0.40,0.24,0.50,5005.30
2,3,0.29,0.40,0.24,0.49,5063.65


Step: Create a function that generates 

In [20]:
import pandas as pd
import numpy as np


def adjust_binary_percentages(df, **column_percentages):
    """
    Adjusts the percentage of 1's in each specified column of a binary DataFrame.

    Args:
    df (pd.DataFrame): DataFrame with binary values.
    column_percentages (dict): A dictionary where keys are column names and values are the new desired percentages of 1's.

    Returns:
    pd.DataFrame: Modified DataFrame.
    """

    for column, percentage in column_percentages.items():
        if column not in df.columns:
            raise ValueError(f"Column {column} not found in DataFrame")

        # Calculate current percentage of 1's
        current_percentage = df[column].mean()

        # Calculate the desired number of 1's
        target_count = int(df.shape[0] * percentage)

        # Find indices where changes are needed
        ones_indices = df[df[column] == 1].index
        zeros_indices = df[df[column] == 0].index

        if target_count > ones_indices.size:  # Need to add more 1's
            change_count = target_count - ones_indices.size
            indices_to_change = np.random.choice(
                zeros_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 1
        else:  # Need to remove some 1's
            change_count = ones_indices.size - target_count
            indices_to_change = np.random.choice(
                ones_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 0

    return df

In [36]:
data_predict_adjust = clean_columns(data_predict.copy())

In [37]:
data_predict_adjust = adjust_binary_percentages(
    df=data_predict_adjust,
    high_blood_pressure=0.9,
    high_cholesterol=0.7,
    diabetes=0.6,
    preventative_services=0.9,
)
data_predict_adjust.rename(
    columns={
        "high_blood_pressure": "High Blood Pressure",
        "high_cholesterol": "High Cholesterol",
        "diabetes": "Diabetes",
        "preventative_services": "Preventative Services",
    },
    inplace=True,
)

In [38]:
# Make predictions (opxtional, to evaluate model)
predictions = model.predict(data_predict_adjust)
predictions_pd = pd.DataFrame(predictions).rename(columns={0: "Predictions"})
predictions_pd

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


,Predictions
0,8722.226562
1,4487.669922
2,7168.820801
3,6342.567871
4,6342.567871
...,...
9995,8722.226562
9996,8722.226562
9997,4487.669922
9998,8141.381348


In [39]:
data_predictions_hospital_id = pd.concat([data["Hospital ID"], predictions_pd], axis=1)

data_predictions_hospital_group = (
    data_predictions_hospital_id.groupby("Hospital ID").mean().reset_index().round(2)
)

noise = np.random.uniform(
    -200, 200, data_predictions_hospital_group["Predictions"].shape
)
data_predictions_hospital_group["Predictions"] = (
    data_predictions_hospital_group["Predictions"] + noise
).round(2)

data_predictions_hospital_group

,Hospital ID,Predictions
0,1,7185.65
1,2,7083.80
2,3,7066.51
